In [1]:
import pandas as pd
import numpy as np
import itertools
import pickle
import torch
import matplotlib.pyplot as plt
from shutil import copyfile
%matplotlib inline
from statistics import mean
from scipy import stats, spatial
from ptflops import get_model_complexity_info
from shutil import copyfile

from main.layout import Layout
from main.models import Deeplab_ASPP_Layout
from main.algorithms import enum_layout, remove_redundancy, enum_layout_wo_rdt, metric_inference, init_S

In [8]:
!bash combined_first_order_launch.sh

Submitted batch job 8512775
Submitted batch job 8512776
Submitted batch job 8512777
Submitted batch job 8512778
Submitted batch job 8512779
Submitted batch job 8512780


In [9]:
!squeue -u lijunzhang

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           8512764  m40-long     V817 lijunzha PD       0:00      1 (Priority)
           8512765  m40-long       V1 lijunzha PD       0:00      1 (Priority)
           8512766  m40-long     V562 lijunzha PD       0:00      1 (Priority)
           8512767  m40-long    V4697 lijunzha PD       0:00      1 (Priority)
           8512768  m40-long    V6539 lijunzha PD       0:00      1 (Priority)
           8512770 1080ti-lo      FO0 lijunzha PD       0:00      1 (Priority)
           8512771 1080ti-lo      FO1 lijunzha PD       0:00      1 (Priority)
           8512772 1080ti-lo      FO2 lijunzha PD       0:00      1 (Priority)
           8512773 1080ti-lo      FO3 lijunzha PD       0:00      1 (Priority)
           8512774 1080ti-lo      FO4 lijunzha PD       0:00      1 (Priority)
           8512775 1080ti-lo      FO0 lijunzha PD       0:00      1 (Priority)
           8512776 1080ti-lo      FO1 lijunzha

# Enum coarse layout B=5

In [10]:
# Enum all layouts for given T (3 for NYUv2) and B (5 for coarse Deeplab_Resnet34)
T = 3 # segment_semantic, normal, depth_zbuffer
B = 6

layout_list = [] 
S0 = init_S(T, B) # initial state
L = Layout(T, B, S0) # initial layout
layout_list.append(L)

enum_layout_wo_rdt(L, layout_list)
len(layout_list)

70

In [4]:
# Simialr layouts
print(layout_list[43])
print(layout_list[44])

[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1}, {0, 2}], [{1}, {0, 2}]]
[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1}, {0, 2}], [{1}, {2}, {0}]]


In [3]:
layout_list

[[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}]],
 [[{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}]],
 [[{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]],
 [[{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]],
 [[{1, 2}, {0}], [{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}], [{0}, {2}, {1}]],
 [[{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{0}, {2}, {1}], [{0}, {2}, {1}]],
 [[{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{1, 2}, {0}], [{0}, {2}, {1}]],
 [[{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}]],
 [[{1}, {0, 2}], [{1}, {2}, {0}], [{1}, {2}, {0}], [{1}, {2}, {0}], [{1}, {2}, {0}]],
 [[{1}, {0, 2}], [{1}, {0, 2}], [{1}, {2}, {0}], [{1}, {2}, {0}], [{1}, {2}, {0}]],
 [[{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {2}, {0}], [{1}, {2}, {0}]],
 [[{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {2}, {0}]],
 [[{2}, {0, 1}], [{2}, {

# Convert coarse layout to fine-grained layout for multi-task model

In [7]:
fined_B = 17
mapping = {0:[0], 1:[1,2,3], 2:[4,5,6,7], 3:[8,9,10,11,12,13], 4:[14,15,16], 5:[17]}

In [15]:
def coarse_to_fined(L, fined_B, mapping):
    S = []
    for idx in range(L.B):
        for times in mapping[idx]:
            S.append(L.state[idx])
    new_L = Layout(L.T, fined_B, S)
    return new_L

In [22]:
coarse_to_fined(layout_list[44], fined_B, mapping)

[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {0, 2}], [{1}, {2}, {0}], [{1}, {2}, {0}], [{1}, {2}, {0}]]

In [18]:
layout_list[44]

[[{0, 1, 2}], [{0, 1, 2}], [{0, 1, 2}], [{1}, {0, 2}], [{1}, {2}, {0}]]

# Convert fined 2task results to coarse

In [2]:
fined_2task_dir = './log/2task_coarse_NYUv2_mobilenet_v4/'
coarse_2task_dir = './log/2task_coarse_NYUv2_mobilenet/'

In [3]:
tasks = ['segment_semantic','normal','depth_zbuffer']
B = 5
# mapping = {0:[0], 1:[1,2], 2:[3], 3:[4], 4:[5,6], 5:[7,8], 6:[9]}
mapping = {0:[0,1,2], 1:[3,4], 2:[5], 3:[6,7], 4:[8], 5:[9]}

for two_task in set(itertools.combinations(tasks, 2)):
    print(two_task)
    
    for idx in range(B+1):
        i = mapping[idx][0]
        log = '_'.join(two_task) + '_b' + str(i) + '.stdout'
        new_log = '_'.join(two_task) + '_b' + str(idx) + '.stdout'
        copyfile(fined_2task_dir + log, coarse_2task_dir + new_log)

('segment_semantic', 'normal')
('segment_semantic', 'depth_zbuffer')
('normal', 'depth_zbuffer')


# Coase version 2 for MobileNet by same flops

In [1]:
from ptflops import get_model_complexity_info
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

Using cache found in /home/lijunzhang/.cache/torch/hub/pytorch_vision_v0.10.0


In [4]:
macs, params = get_model_complexity_info(model, (3,321,321), as_strings=True, print_per_layer_stat=True, verbose=False)

MobileNetV2(
  3.505 M, 100.000% Params, 0.716 GMac, 100.000% MACs, 
  (features): Sequential(
    2.224 M, 63.451% Params, 0.715 GMac, 99.821% MACs, 
    (0): ConvBNActivation(
      0.001 M, 0.026% Params, 0.025 GMac, 3.475% MACs, 
      (0): Conv2d(0.001 M, 0.025% Params, 0.022 GMac, 3.127% MACs, 3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(0.0 M, 0.002% Params, 0.002 GMac, 0.232% MACs, 32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(0.0 M, 0.000% Params, 0.001 GMac, 0.116% MACs, inplace=True)
    )
    (1): InvertedResidual(
      0.001 M, 0.026% Params, 0.024 GMac, 3.359% MACs, 
      (conv): Sequential(
        0.001 M, 0.026% Params, 0.024 GMac, 3.359% MACs, 
        (0): ConvBNActivation(
          0.0 M, 0.010% Params, 0.01 GMac, 1.390% MACs, 
          (0): Conv2d(0.0 M, 0.008% Params, 0.007 GMac, 1.042% MACs, 32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
 

In [2]:
model

MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen